In [ ]:
#r "nuget:System.Reactive"
#r "nuget:System.Reactive.Linq"

In [ ]:
using System.Reactive;
using System.Reactive.Concurrency;
using System.Reactive.Disposables;
using System.Reactive.Joins;
using System.Reactive.Linq;
using System.Reactive.PlatformServices;
using System.Reactive.Subjects;
using System.Reactive.Threading.Tasks;
using System.Threading;
using System.Threading.Tasks;

// display output
var output = display("idle.");
var outputSensor = display("<sensor>");
var outputLight = display("<light>");

// hass-net-2
// timer
var delay = TimeSpan.FromSeconds(10);
bool lightsOn = false;

var controlList = new List<string>();
var controlChannel = controlList.ToObservable();

// sensor
var interval = TimeSpan.FromSeconds(6);
var schedular = CurrentThreadScheduler.Instance;
var compdisb = new CompositeDisposable();
var sdisposable = new ScheduledDisposable(schedular, compdisb);

using (sdisposable) {
var observableSensor1 = Observable.Interval(interval, schedular);
var observableSensor = observableSensor1.Publish();
IDisposable sensorOn2 = observableSensor
	.Select(t => DateTimeOffset.Now)
	.TakeUntil(controlChannel.Where(m => m == "STOP"))
	.Timestamp()
	.Subscribe((onCount) => {
		outputSensor.Update("sensor-on-2 " + onCount.ToString());
		controlList.Add("RESET");
		output.Update("timer-reset" + onCount.ToString());
		controlList.Clear();
	});
	compdisb.Add(sensorOn2);
IDisposable sensorOn1 = observableSensor
	.Select(t => DateTimeOffset.Now)
	.Timestamp()
	.TakeUntil(controlChannel.Where(m => m == "STOP" || m == "RESET"))
	.Do((onCount) => {
		outputSensor.Update("sensor-on " + onCount.ToString());
	})
	.Do((onCount) => {
		output.Update("timer-on" + onCount.ToString());
	})
	.TakeUntil(DateTimeOffset.Now.AddSeconds(10))
	.Subscribe((onCount) => {
		output.Update("light on" + onCount.ToString());
		lightsOn = true;
	});
	compdisb.Add(sensorOn1);
	
	IDisposable oc = observableSensor.Connect();
	compdisb.Add(oc);


await Task.Delay (30000);
controlList.Add("STOP");


}

timer-reset18/08/2022 00:00:52 +02:00@17/08/2022 22:00:52 +00:00

sensor-on-2 18/08/2022 00:00:52 +02:00@17/08/2022 22:00:52 +00:00

<light>

Error: System.OperationCanceledException: Command :SubmitCode: using System.Reactive;
using System.Reactive.Conc ... cancelled.

In [ ]:
using System.Reactive;
using System.Reactive.Linq;
using System.Reactive.Concurrency;

var output = display("Counting...");

var sub = Observable
    .Interval(TimeSpan.FromSeconds(.5), CurrentThreadScheduler.Instance)    
    .Take(10) 
    .Subscribe(i => output.Update(i));

9

In [ ]:
var fruitOutput = display("Let's get some fruit!");
var basket = new [] {"apple", "orange", "coconut", "pear", "peach"};

foreach (var fruit in basket)
{
    System.Threading.Thread.Sleep(1000);
    fruitOutput.Update($"I have 1 {fruit}.");    
}

System.Threading.Thread.Sleep(1000);

fruitOutput.Update(basket);

In [ ]:
using System.Reactive;
using System.Reactive.Concurrency;
using System.Reactive.Disposables;
using System.Reactive.Joins;
using System.Reactive.Linq;
using System.Reactive.PlatformServices;
using System.Reactive.Subjects;
using System.Reactive.Threading.Tasks;
using System.Threading;
using System.Threading.Tasks;

// display output
var outputState = display("idle.");
var outputSensor = display("nothing detected");
var outputLight = display("lights-off");
var log = new List<string>();
var outputLog = display(log.ToString());

// hass-net-2
// timer
var delay = TimeSpan.FromSeconds(10);
long lightsOn = 0;

var controlList = new List<string>();
var controlChannel = controlList.ToObservable();

// sensor
var interval = TimeSpan.FromSeconds(6);
var timeoutMotionLight = TimeSpan.FromSeconds(10);
var schedular = CurrentThreadScheduler.Instance;

var observableSensorInterval = Observable.Interval(interval, schedular).TakeUntil(DateTimeOffset.Now.AddSeconds(13));
var observableSensor1 = observableSensorInterval.Publish();
var motionDetected = observableSensor1
	.Timeout(timeoutMotionLight)
	.Catch<long, TimeoutException>(tx => {
		// handle timeout
		outputLight.Update("lights-off-1");

		// swallow exception
		return Observable.Return((long)-1);
	})
	.Timestamp()
	.TimeInterval()
	.Repeat()
	.Subscribe(onNext: s => {
		log.Add("e: " + s.Display());
		// received Motion detected event => turn lights on
		lightsOn =+ 1;
		outputLight.Update("lights-ON" + lightsOn);
		
});

var motionTimeout = observableSensor1
	.Timeout(timeoutMotionLight)
	// .Catch<long, TimeoutException>(tx => {
	// 	// handle timeout
	// 	outputLight.Update("lights-off");

	// 	// swallow exception
	// 	return Observable.Return((long)-1);
	// })
	.Timestamp()
	.TimeInterval()
	.Repeat()
	.Subscribe(onNext: _ =>  outputState.Update("tn"),
		onError: e => {
		log.Add("e: " + e.Display());
		// received Motion detected event => turn lights on
		
		outputLight.Update("lights-off-2");
		
});
	
observableSensor1.Connect();	

await Task.Delay (30000);
controlList.Add("STOP");




tn

nothing detected

lights-off-2

System.Collections.Generic.List`1[System.String]

Value,Interval
"{ 0@18/08/2022 20:52:08 +00:00: Value: 0, Timestamp: 2022-08-18 20:52:08Z }",00:00:06.0371489


Value,Interval
"{ 1@18/08/2022 20:52:14 +00:00: Value: 1, Timestamp: 2022-08-18 20:52:14Z }",00:00:05.9982620


TargetSite,Message,Data,InnerException,HelpLink,Source,HResult,StackTrace
<null>,The operation has timed out.,[ ],<null>,<null>,<null>,-2146233083,<null>


Error: System.OperationCanceledException: Command :SubmitCode: using System.Reactive;
using System.Reactive.Conc ... cancelled.

In [ ]:
#!share --from csharp output


In [ ]:
#!share --from csharp output


In [ ]:
#!share --from javascript webview
